In [ ]:
import numpy

__Don't forget to download the data file [from here](http://sandbox.hlt.bme.hu/~gaebor/ea_anyag/python_nlp/)__

### Task 1

In [ ]:
word_dict = {}
pos_dict = {}
with open("umbc.casesensitive.word_pos.1M.txt", encoding="utf8", errors="ignore") as f:
    for line in f:
        split_line = line.strip().split("\t")
        if len(split_line) == 2:
            word, pos = line.strip().split("\t")
            if word not in word_dict:
                index = len(word_dict)
                word_dict[word] = index
            if pos not in pos_dict:
                index = len(pos_dict)
                pos_dict[pos] = index
index_to_word = {i: w for w, i in word_dict.items()}
index_to_pos = {i: p for p, i in pos_dict.items()}

### Task 2

In [ ]:
M = numpy.zeros((len(word_dict), len(pos_dict)), dtype=int)
N = numpy.zeros((len(pos_dict), len(pos_dict)), dtype=int)

pos_list = []
with open("umbc.casesensitive.word_pos.1M.txt", encoding="utf8", errors="ignore") as f:
    for line in f:
        split_line = line.strip().split("\t")
        if len(split_line) == 2:
            word, pos = line.strip().split("\t")
            pos_list.append(pos)
            M[word_dict[word], pos_dict[pos]] += 1
            
            if len(pos_list) == 2:
                N[pos_dict[pos_list[0]], pos_dict[pos_list[1]]] += 1
                del pos_list[0]

In [ ]:
N

In [ ]:
P1 = M / M.sum(axis=0)[None, :].astype(float)
P2 = N / N.sum(axis=0)[None, :].astype(float)
P2

### Task 3

In [ ]:
def viterbi_step(previous, v, word):
    best = 0.0
    for w in pos_dict:
        p = previous[pos_dict[w]] * P2[pos_dict[w], pos_dict[v]] * P1[word_dict[word], pos_dict[v]]
        if p > best:
            best = p
    return best

def viterbi_step_index(previous, v, word):
    """the same but with integer IDs, not strings"""
    best = 0.0
    for w in range(len(pos_dict)):
        p = previous[w] * P2[w, v] * P1[word, v]
        if p > best:
            best = p
    return best

In [ ]:
def viterbi(words):
    pi = numpy.zeros((len(pos_dict), len(words)), dtype=float)
    for v in pos_dict:
        pi[pos_dict[v], 0] = P1[word_dict[words[0]], pos_dict[v]]
    
    for k in range(1, len(words)):
        for v in pos_dict:
            value = viterbi_step(pi[:, k-1], v, words[k])
            pi[pos_dict[v], k] = value

    best_ending_v = 0
    for v in range(len(pos_dict)):
            if pi[v, -1] > pi[best_ending_v, -1]:
                best_ending_v = v
    return pi[best_ending_v, -1]

def viterbi_index(words):
    """the same but with integer IDs, not strings"""
    pi = numpy.zeros((len(pos_dict), len(words)), dtype=float)
    for v in range(len(pos_dict)):
        pi[v, 0] = P1[word_dict[words[0]], v]
    
    for k in range(1, len(words)):
        for v in range(len(pos_dict)):
            value = viterbi_step_index(pi[:, k-1], v, word_dict[words[k]])
            pi[v, k] = value

    best_ending_v = 0
    for v in range(len(pos_dict)):
        if pi[v, -1] > pi[best_ending_v, -1]:
            best_ending_v = v
    return pi[best_ending_v, -1]

In [ ]:
viterbi("The cat sat on the desk .".split())
viterbi_index("The cat sat on the desk .".split())

### Task 4
Add the backtracking with an extra table, which stores the argmax, not the max value.

In [ ]:
def viterbi_step(previous, v, word):
    """
    using indices
    returning max value and argmax
    """
    
    best = 0.0
    best_choice = 0
    for w in range(len(pos_dict)):
        p = previous[w] * P2[w, v] * P1[word, v]
        if p > best:
            best = p
            best_choice = w
    return best, best_choice

def viterbi(words):
    pi = numpy.zeros((len(pos_dict), len(words)), dtype=float)
    choices = numpy.zeros((len(pos_dict), len(words)), dtype="uint32")
    
    for v in range(len(pos_dict)):
        pi[v, 0] = P1[word_dict[words[0]], v]
    
    for k in range(1, len(words)):
        for v in range(len(pos_dict)):
            value, choice = viterbi_step(pi[:, k-1], v, word_dict[words[k]])
            pi[v, k] = value
            choices[v, k] = choice

    best_ending_v = 0
    for v in range(len(pos_dict)):
        if pi[v, -1] > pi[best_ending_v, -1]:
            best_ending_v = v
    
    optimal = [best_ending_v]
    
    # backtracking
    for k in range(len(words)-1, 0, -1):
        # prepend optimal sequence
        optimal = [choices[optimal[0], k]] + optimal
        
    return pi[best_ending_v, -1], [index_to_pos[p] for p in optimal]


In [ ]:
viterbi("The dog sat on the cat .".split())